In [89]:
import numpy
# scipy.sepcial for the sigmoid function expit()
import scipy.special
# neural network class definition
import matplotlib.pyplot
%matplotlib inline

class neuralNetwork:
    #initialise th neural network
    def __init__(self,inputnodes,hiddennodes,outputnodes,learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        self.lr = learningrate

        #link weight matrices, wit and who
        #weights inside the arrays are w_i_j, where link is from node i to node j in next layer
        #w11 w21 etc
        self.wih = (numpy.random.rand(self.hnodes,self.inodes)-0.5)
        self.who = (numpy.random.rand(self.onodes,self.hnodes)-0.5)
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
        
    #train the neural network
    def train(self, inputs_list, targets_list):
        
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list,ndmin=2).T
        targets = numpy.array(targets_list,ndmin=2).T

        hidden_inputs = numpy.dot(self.wih,inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = numpy.dot(self.who,hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        #calculate error. this is colum vec
        output_errors = targets - final_outputs

        #hidden layer error is the output_errors, split by weights, recombined at hidden notes
        hidden_errors = numpy.dot(self.who.T, output_errors)

        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass
        
    #query the neural network
    def query(self,inputs_lists):
        inputs = numpy.array(inputs_lists,ndmin=2).T
        hidden_inputs = numpy.dot(self.wih,inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = numpy.dot(self.who,hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs

In [90]:
input_nodes = 784
hidden_nodes = 600
output_nodes = 10

learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)



In [91]:
# load the minst training data csv file into a list
training_data_file = open("./mnist_train_100.csv","r")
training_data_list = training_data_file.readlines()
training_data_file.close()


In [92]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 10

for e in range (epochs):
  # go through all records in the training data set
  for record in training_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # scale and shift the inputs
    inputs = numpy.asfarray(all_values[1:])/255.0*0.99+0.01
    targets = numpy.zeros(output_nodes)+0.01
    # set label
    targets[int(all_values[0])] = 0.99
    n.train(inputs,targets)
    pass
  pass




In [93]:
# load the mnist test data CSV file into a list
test_data_file = open("mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [94]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [95]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print(scorecard_array)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

[1 1 1 1 1 1 0 0 0 0]
performance =  0.6
